# Homomorphic


In [ ]:
# import logging
# import numpy as np

# # Homomorphic filter class
# class HomomorphicFilter:
#     """Homomorphic filter implemented with diferents filters and an option to an external filter.
    
#     High-frequency filters implemented:
#         butterworth
#         gaussian
#     Attributes:
#         a, b: Floats used on emphasis filter:
#             H = a + b*H
        
#         .
#     """

#     def __init__(self, a = 0.5, b = 1.5):
#         self.a = float(a)
#         self.b = float(b)

#     # Filters
#     def __butterworth_filter(self, I_shape, filter_params):
#         P = I_shape[0]/2
#         Q = I_shape[1]/2
#         U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
#         Duv = (((U-P)**2+(V-Q)**2)).astype(float)
#         H = 1/(1+(Duv/filter_params[0]**2)**filter_params[1])
#         return (1 - H)

#     def __gaussian_filter(self, I_shape, filter_params):
#         P = I_shape[0]/2
#         Q = I_shape[1]/2
#         H = np.zeros(I_shape)
#         U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
#         Duv = (((U-P)**2+(V-Q)**2)).astype(float)
#         H = np.exp((-Duv/(2*(filter_params[0])**2)))
#         return (1 - H)

#     # Methods
#     def __apply_filter(self, I, H):
#         H = np.fft.fftshift(H)
#         I_filtered = (self.a + self.b*H)*I
#         return I_filtered

#     def filter(self, I, filter_params, filter='butterworth', H = None):
#         """
#         Method to apply homormophic filter on an image
#         Attributes:
#             I: Single channel image
#             filter_params: Parameters to be used on filters:
#                 butterworth:
#                     filter_params[0]: Cutoff frequency 
#                     filter_params[1]: Order of filter
#                 gaussian:
#                     filter_params[0]: Cutoff frequency
#             filter: Choose of the filter, options:
#                 butterworth
#                 gaussian
#                 external
#             H: Used to pass external filter
#         """

#         #  Validating image
#         if len(I.shape) is not 2:
#             raise Exception('Improper image')

#         # Take the image to log domain and then to frequency domain 
#         I_log = np.log1p(np.array(I, dtype="float"))
#         I_fft = np.fft.fft2(I_log)

#         # Filters
#         if filter=='butterworth':
#             H = self.__butterworth_filter(I_shape = I_fft.shape, filter_params = filter_params)
#         elif filter=='gaussian':
#             H = self.__gaussian_filter(I_shape = I_fft.shape, filter_params = filter_params)
#         elif filter=='external':
#             print('external')
#             if len(H.shape) is not 2:
#                 raise Exception('Invalid external filter')
#         else:
#             raise Exception('Selected filter not implemented')
        
#         # Apply filter on frequency domain then take the image back to spatial domain
#         I_fft_filt = self.__apply_filter(I = I_fft, H = H)
#         I_filt = np.fft.ifft2(I_fft_filt)
#         I = np.exp(np.real(I_filt))-1
#         return np.uint8(I)
# # End of class HomomorphicFilter

# if __name__ == "__main__":
#     import cv2
#     # Code parameters
#     path_in = '/home/images/in/'
#     path_out = '/home/images/out/'
#     img_path = 'original.jpg'

#     # Derived code parameters
#     img_path_in = path_in + img_path
#     img_path_out = path_out + 'filtered.png'

#     # Main code
#     img = cv2.imread(img_path_in)[:, :, 0]
#     homo_filter = HomomorphicFilter(a = 0.75, b = 1.25)
#     img_filtered = homo_filter.filter(I=img, filter_params=[30,2])
#     cv2.imwrite(img_path_out, img_filtered)

# Fruit Fly Algorithm

In [1]:
import sys
import random
import math
#import numpy as np
import pandas as pd
#from sklearn.datasets import load_breast_cancer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.utils import shuffle

def readDataset():
    dataset = pd.read_csv('breastcancer.csv', header=None)
    dataset[10] = dataset[10].replace(2,0)
    dataset[10] = dataset[10].replace(4,1)
    dataset = dataset[dataset[6]!='?']
    #dataset.replace('?', -9999, inplace=True)
    #dataset = dataset.applymap(np.int64)
    data = dataset.iloc[:, 1:-1].values
    target = dataset.iloc[:, -1].values
    return (data, target)

def readDiabetesDataset():
    dataset = pd.read_csv('diabetes.csv')
    data = dataset.iloc[:,:-1].values
    target = dataset.iloc[:,-1].values
    return (data,target)

def readParkinsonsDataset():
    dataset = pd.read_csv('parkinsons.csv')
    dataset = dataset.drop('name',1)
    data = dataset.drop('status',1).values
    target = dataset.iloc[:,16].values
    return (data, target)

def readThyroidDataset():
    dataset = pd.read_csv('thyroid.csv', header=None)
    dataset = shuffle(dataset)
    target = dataset.iloc[:,0].values
    data = dataset.iloc[:, 1:].values
    return (data, target)

class fly:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2

def fitness(data_scaled, target, x,y):
    clf = svm.SVC(kernel='rbf', C=x, gamma=y)
    scores = cross_val_score(clf,data_scaled, target, cv=10)
    #print(scores)   
    #print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    #print(scores)
    return scores.mean()

def run(data_scaled, target, population, x_axis1, y_axis1, bestSmell, x_axis2, y_axis2):
    
    flies = []
    
    for i in range(0,population):
        
        xi1 = x_axis1 + random.uniform(-1,1)
        yi1 = y_axis1 + random.uniform(-1,1)
        xi2 = x_axis2 + random.uniform(-1,1)
        yi2 = y_axis2 + random.uniform(-1,1)
        
        
        flies.append(fly(xi1,yi1,xi2,yi2))
        
    minSmell =  -sys.maxsize
    bestIndex = -1
    
    for currFly in flies:
        
        d1 = math.sqrt(math.pow(currFly.x1,2)+math.pow(currFly.y1,2))
        d2 = math.sqrt(math.pow(currFly.x2,2)+math.pow(currFly.y2,2))
        
        d1_inv = round(1/d1,2)
        d2_inv = round(1/d2,2)
        d1_inv = min(d1_inv,math.pow(2,15))
        d1_inv = max(d1_inv,math.pow(2,-5))
        d2_inv = min(d2_inv,2)
        d2_inv = max(d2_inv,math.pow(2,-15))
        

        
        smell = fitness(data_scaled, target, d1_inv, d2_inv)
        if(minSmell < smell):
            minSmell = smell
            bestIndex = flies.index(currFly)
    
    bestSmell = minSmell
    x_axis1 = flies[bestIndex].x1
    y_axis1 = flies[bestIndex].y1
    x_axis2 = flies[bestIndex].x2
    y_axis2 = flies[bestIndex].y2
    
    return (x_axis1, y_axis1, bestSmell, x_axis2, y_axis2)


def main():
    population = 10
    x_axis1 = random.uniform(0,1)
    y_axis1 = random.uniform(0,1)
    x_axis2 = random.uniform(0,1)
    y_axis2 = random.uniform(0,1)
    bestSmell = sys.maxsize
    max_iter = 250
    
    data, target = readDiabetesDataset()
    data_scaled = preprocessing.scale(data)
    
    for i in range(max_iter):
        print(f'Running iteration {i+1} ...')
        x_axis1, y_axis1, bestSmell, x_axis2, y_axis2 = run(data_scaled, target, population, x_axis1, y_axis1, bestSmell, x_axis2, y_axis2)
        print(round((1/math.sqrt(math.pow(x_axis1,2)+math.pow(y_axis1,2))),2), round((1/math.sqrt(math.pow(x_axis2,2)+math.pow(y_axis2,2))),2), bestSmell)

main()

# Multiple Morphological Gradients (use for one image and check)

In [2]:
# import cv2 
# import numpy as np 
 
# # return video from the first webcam on your computer. 
# screenRead = cv2.VideoCapture(0)
 
# # loop runs if capturing has been initialized.
# while(1):
#     # reads frames from a camera
#     _, image = screenRead.read()
     
#     # Converts to HSV color space, OCV reads colors as BGR
#     # frame is converted to hsv
#     hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
     
#     # defining the range of masking
#     blue1 = np.array([110, 50, 50])
#     blue2 = np.array([130, 255, 255])
     
#     # initializing the mask to be
#     # convoluted over input image
#     mask = cv2.inRange(hsv, blue1, blue2)
 
#     # passing the bitwise_and over
#     # each pixel convoluted
#     res = cv2.bitwise_and(image, image, mask = mask)
     
#     # defining the kernel i.e. Structuring element
#     kernel = np.ones((5, 5), np.uint8)
     
#     # defining the gradient function
#     # over the image and structuring element
#     gradient = cv2.morphologyEx(mask, cv2.MORPH_GRADIENT, kernel)
    
#     # The mask and closing operation
#     # is shown in the window
#     cv2.imshow('Gradient', gradient)
     
#     # Wait for 'a' key to stop the program
#     if cv2.waitKey(1) & 0xFF == ord('a'):
#         break
 

# High Frequency Estimation

In [3]:
import numpy as np
from scipy.fftpack import fft2, ifft2, fftshift
import imageio
from .base import BaseAlgorithm
import src.utils as pu


class HEF(BaseAlgorithm):
    def __init__(self, filename, results_path):
        self.filename = filename
        self.get_input()
        self.results_path = results_path

    def get_input(self):
        print(
            "Select D0 value for High cut (1 to 90): ")
        self.d0v = int(input())
        assert 1 <= self.d0v <= 90

    def run(self):
        '''Runs the algorithm for the image.'''
        image = imageio.imread(self.filename)

        if len(image.shape) == 3:
            img_grayscale = pu.to_grayscale(image)
        img = pu.normalize(np.min(img_grayscale), np.max(image), 0, 255,
                                        img_grayscale)
        # HF part
        img_fft = fft2(img)  # img after fourier transformation
        img_sfft = fftshift(img_fft)  # img after shifting component to the center

        m, n = img_sfft.shape
        filter_array = np.zeros((m, n))

        for i in range(m):
            for j in range(n):
                filter_array[i, j] = 1.0 - np.exp(- ((i-m / 2.0) ** 2 + (j-n / 2.0) ** 2) / (2 * (self.d0v ** 2)))
        k1 = 0.5
        k2 = 0.75
        high_filter = k1 + k2*filter_array

        img_filtered = high_filter * img_sfft
        img_hef = np.real(ifft2(fftshift(img_filtered)))  # HFE filtering done

        # HE part
        # Building the histogram
        hist, bins = pu.histogram(img_hef)
        # Calculating probability for each pixel
        pixel_probability = hist / hist.sum()
        # Calculating the CDF (Cumulative Distribution Function)
        cdf = np.cumsum(pixel_probability)
        cdf_normalized = cdf * 255
        hist_eq = {}
        for i in range(len(cdf)):
            hist_eq[bins[i]] = int(cdf_normalized[i])

        for i in range(m):
            for j in range(n):
                image[i][j] = hist_eq[img_hef[i][j]]

        return image.astype(np.uint8)

# For denoising -- NL Means

In [4]:
# import numpy as np
# import cv2 as cv
# from matplotlib import pyplot as plt
# cap = cv.VideoCapture('vtest.avi')
# # create a list of first 5 frames
# img = [cap.read()[1] for i in range(5)]
# # convert all to grayscale
# gray = [cv.cvtColor(i, cv.COLOR_BGR2GRAY) for i in img]
# # convert all to float64
# gray = [np.float64(i) for i in gray]
# # create a noise of variance 25
# noise = np.random.randn(*gray[1].shape)*10
# # Add this noise to images
# noisy = [i+noise for i in gray]
# # Convert back to uint8
# noisy = [np.uint8(np.clip(i,0,255)) for i in noisy]
# # Denoise 3rd frame considering all the 5 frames
# dst = cv.fastNlMeansDenoisingMulti(noisy, 2, 5, None, 4, 7, 35)
# plt.subplot(131),plt.imshow(gray[2],'gray')
# plt.subplot(132),plt.imshow(noisy[2],'gray')
# plt.subplot(133),plt.imshow(dst,'gray')
# plt.show()